##### Imports

In [1]:
import sys
from pathlib import Path
import warnings
from torch.utils.data import Dataset

import warnings
import pandas as pd
warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', 1000)
pd.set_option('display.max_rows', 1000)

import pickle
from pathlib import Path
from scr.dataset import *
# from scr.game import *
from scr.feature_engineering import *
# from scr.plot_utils import *
import gc
from scr.utils import print_scenarios

from pathlib import Path
import random
from collections import Counter, defaultdict
import pickle
from tqdm import tqdm
from torch.utils.data import Dataset

from scr.utils import read_words, save_words_to_file

import sys
# Custom library paths
sys.path.extend(['../', './scr'])

from scr.utils import set_seed
from scr.utils import read_words

set_seed(42)

import torch
import torch.nn as nn
from pathlib import Path
import random

torch.set_float32_matmul_precision('medium')

# Device configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# # Read and Shuffle Word List
word_list = read_words('data/words_250000_train.txt') # , limit=10000)
# word_list = read_words('data/250k.txt', limit=10000)

random.shuffle(word_list)

# Calculate Frequencies and Max Word Length
word_frequencies = calculate_word_frequencies(word_list)
char_frequency = calculate_char_frequencies(word_list)
max_word_length = max(len(word) for word in word_list)

##### Data Path

In [2]:
import shutil
from pathlib import Path

NUM_STRATIFIED_SAMPLES = 100 # This will be overwritten by Papermill

# Define the base directory and the paths for training and validation parquet files
base_dataset_dir = Path("/media/sayem/510B93E12554BBD1/dataset/")

stratified_samples_dir = base_dataset_dir / str(NUM_STRATIFIED_SAMPLES)

parquet_train_path = stratified_samples_dir / 'train_parquets'
parquet_valid_path = stratified_samples_dir / 'valid_parquets'

# Function to delete and recreate a directory
def recreate_directory(path):
    if path.exists():
        shutil.rmtree(path)  # Delete the directory and its contents
    path.mkdir(parents=True)  # Create the directory

# Recreate the train and valid directories
recreate_directory(parquet_train_path)
recreate_directory(parquet_valid_path)

print(f"Directories '{parquet_train_path}' and '{parquet_valid_path}' have been recreated.")

Directories '/media/sayem/510B93E12554BBD1/dataset/100/train_parquets' and '/media/sayem/510B93E12554BBD1/dataset/100/valid_parquets' have been recreated.


In [3]:
len(word_list)

227300

##### Testing Data

In [4]:
# Define the total number of words and the number of test samples
from scr.custom_sampler import *
NUM_TEST_SAMPLES = 10_000

# Assuming 'word_list' contains the 250,000 words
# First, separate 10,000 words for the final testing set
testing_words = stratified_sample_by_length_and_uniqueness(
    word_list, 
    NUM_TEST_SAMPLES
)

# Define the file path for saving the testing words
testing_words_file_path = stratified_samples_dir / "testing_words.txt"

# Save the testing words to a file
with open(testing_words_file_path, 'w') as file:
    for word in testing_words:
        file.write(word + '\n')

print(f"Testing words saved in {testing_words_file_path}")

# Now, remove these testing samples from the original word list
remaining_words = [word for word in word_list if word not in testing_words]

Testing words saved in /media/sayem/510B93E12554BBD1/dataset/100/testing_words.txt


##### Stratified Sample Generation

In [5]:
## we are taking starified samples from train_words

from scr.custom_sampler import \
    stratified_sample_by_length_and_frequency, \
        stratified_sample_by_length, stratified_sample_by_length_and_uniqueness

print(f'Number of Statrified samples: {NUM_STRATIFIED_SAMPLES}')

# sampled_words_by_length_and_frequency \
#     = stratified_sample_by_length_and_frequency(train_words, \
#     word_frequencies, \
#     NUM_STRATIFIED_SAMPLES)

sampled_words_by_length = stratified_sample_by_length_and_uniqueness(remaining_words, \
    NUM_STRATIFIED_SAMPLES)

print(len(sampled_words_by_length))

Number of Statrified samples: 100
207


##### Intial State Simulation Testing

In [6]:
from scr.game import *

In [7]:
# word = "mississippi"
word = "mythopoetize"
# word = "cat"

initial_states = process_word_for_six_states(word)

initial_states

{'allMasked': '____________',
 'early': '___h________',
 'quarterRevealed': '_y_h________',
 'midRevealed': '_ytho_o_t___',
 'midLateRevealed': '_ythopoet__e',
 'lateRevealed': 'mythopoeti_e',
 'nearEnd': 'mythopoeti_e'}

In [8]:
len(initial_states)

7

In [9]:
initial_states

{'allMasked': '____________',
 'early': '___h________',
 'quarterRevealed': '_y_h________',
 'midRevealed': '_ytho_o_t___',
 'midLateRevealed': '_ythopoet__e',
 'lateRevealed': 'mythopoeti_e',
 'nearEnd': 'mythopoeti_e'}

##### Dataset Generation: Simulation

In [10]:
word = "mississippi"

initial_states = process_word_for_six_states(word)

# print(initial_states)
# Print generated initial states
print("Generated Initial States: ", initial_states)

Generated Initial States:  {'allMasked': '___________', 'early': '__ss_ss____', 'quarterRevealed': '__ss_ss____', 'midRevealed': '_ississi__i', 'midLateRevealed': '_ississippi', 'lateRevealed': '_ississippi', 'nearEnd': '_ississippi'}


In [11]:
from scr.game import simulate_game_progress, \
    play_game_with_a_word, process_word

# Example word and initial state
# Example usage
word = "mississippi"
# word = "cat"

initial_states = process_word_for_six_states(word)

# print(initial_states)
# Print generated initial states
print("Generated Initial States:")
for state_name, initial_state in initial_states.items():
    # Simulate the game
    print(initial_state)
    print(f"For initial state: {initial_state}")
    won, game_progress = simulate_game_progress(
        model=None,  # Assuming model is not used in this example
        word=word, 
        initial_state=initial_state, 
        char_frequency={},  # Assuming char_frequency is not used in this example
        max_word_length=len(word), 
        device=None,  # Assuming device is not used in this example
        max_attempts=6, 
        normalize=True,
        difficulty="medium", 
        outcome_preference='win'
    )

    # Display game progress
    for step in game_progress:
        print(f"Guessed: '{step[0]}', New State: '{step[1]}', Correct: {step[2]}")

        # break

    # break

    # print("Game Result:", "Won" if won else "Lost")

Generated Initial States:
___________
For initial state: ___________
Guessed: 'm', New State: 'm__________', Correct: True
Guessed: 'p', New State: 'm_______pp_', Correct: True
Guessed: 'y', New State: 'm_______pp_', Correct: False
Guessed: 's', New State: 'm_ss_ss_pp_', Correct: True
Guessed: 'i', New State: 'mississippi', Correct: True
________pp_
For initial state: ________pp_
Guessed: 'i', New State: '_i__i__ippi', Correct: True
Guessed: 's', New State: '_ississippi', Correct: True
Guessed: 't', New State: '_ississippi', Correct: False
Guessed: 'h', New State: '_ississippi', Correct: False
Guessed: 'z', New State: '_ississippi', Correct: False
Guessed: 'm', New State: 'mississippi', Correct: True
________pp_
For initial state: ________pp_
Guessed: 'b', New State: '________pp_', Correct: False
Guessed: 'm', New State: 'm_______pp_', Correct: True
Guessed: 's', New State: 'm_ss_ss_pp_', Correct: True
Guessed: 'i', New State: 'mississippi', Correct: True
_i__i__ippi
For initial state:

##### Writing Parquet

In [12]:
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
from tqdm import tqdm
from sklearn.model_selection import train_test_split

# Assuming the function 'process_word_for_six_states' is defined elsewhere
from scr.game import simulate_game_progress, process_word_for_six_states

def process_batch_to_parquet(batch, file_path, start_game_counter):
    game_counter = start_game_counter
    data_for_parquet = []

    for game_data in batch:
        word, state_name, initial_state, difficulty, outcome, won, game_progress = game_data
        if not game_progress:
            continue

        final_state = game_progress[-1][1]
        guessed_states = [initial_state] + [state for _, state, _ in game_progress]
        guessed_letters = [letter for letter, _, _ in game_progress]

        data_for_parquet.append({
            'game_id': game_counter,
            'word': word,
            'initial_state': initial_state,
            'final_state': final_state,
            'guessed_states': ','.join(guessed_states[:-1]),
            'guessed_letters': ','.join(guessed_letters),
            'game_state': state_name,
            'difficulty': difficulty,
            'outcome': outcome,
            'word_length': len(word),
            'won': won
        })

        game_counter += 1

    df = pd.DataFrame(data_for_parquet)
    table = pa.Table.from_pandas(df)
    pq.write_to_dataset(table, root_path=file_path, compression='snappy')

    return game_counter

def generate_batch_for_word(word):
    batch = []
    initial_states = process_word_for_six_states(word)
    games_generated = 0  # Counter for games generated

    for state_name, initial_state in initial_states.items():
        for difficulty in ["easy", "medium", "hard"]:
            for outcome in ["win", "lose"]:
                won, game_progress = simulate_game_progress(
                    model=None, word=word, initial_state=initial_state,
                    char_frequency={}, max_word_length=len(word),
                    device=None, max_attempts=6, normalize=True,
                    difficulty=difficulty, outcome_preference=outcome
                )
                batch.append((word, state_name, initial_state, difficulty, outcome, won, game_progress))
                games_generated += 1  # Increment game counter

    return batch, games_generated


def main_execution(words, parquet_train_path, parquet_valid_path, test_size=0.20):
    train_game_counter = 0
    valid_game_counter = 0
    games_per_word = 6 * 3 * 2  # 6 states, 3 difficulties, 2 outcomes

    train_words, valid_words = train_test_split(words, test_size=test_size)

    # Process each word set
    for word_set, path in [(train_words, parquet_train_path), (valid_words, parquet_valid_path)]:
        total_words_processed = 0
        total_games_generated = 0

        for word in tqdm(word_set, desc="Processing Words"):
            batch, games_generated = generate_batch_for_word(word)

            # Update word count and game count
            total_words_processed += 1
            total_games_generated += games_generated

            # Update game counters
            if word_set is train_words:
                train_game_counter += games_generated
                process_batch_to_parquet(batch, path, train_game_counter)
            else:
                valid_game_counter += games_generated
                process_batch_to_parquet(batch, path, valid_game_counter)

    print(f"Final Total games processed in training set: {train_game_counter}")
    print(f"Final Total games processed in validation set: {valid_game_counter}")

# Execute the main function
main_execution(sampled_words_by_length, parquet_train_path, parquet_valid_path, test_size=0.20)

Processing Words: 100%|██████████| 42/42 [00:00<00:00, 57.27it/s]

Final Total games processed in training set: 6930
Final Total games processed in validation set: 1764


##### Checking Train

In [13]:
import pandas as pd
from pathlib import Path

# Assuming parquet_train_path is already defined as a Path object
# If not, define it here
# parquet_train_path = Path('path_to_your_train_parquet_directory')

# Use glob to find all Parquet files in the folder
parquet_files = parquet_train_path.glob('*.parquet')

# Count the number of files
file_count = sum(1 for _ in parquet_files)

print(f"Number of Parquet files: {file_count}")

Number of Parquet files: 165


In [14]:
import pandas as pd
from pathlib import Path


# Find all Parquet files in the directory
parquet_files = list(parquet_train_path.glob('*.parquet'))

if parquet_files:
    total_game_sequences = 0

    # Iterate over each file and sum the number of game sequences
    for file in parquet_files:
        df = pd.read_parquet(file)
        total_game_sequences += len(df)

    print(f"Total number of game sequences across all files: {total_game_sequences}")
else:
    print("No Parquet files found in the specified directory.")


Total number of game sequences across all files: 6894


In [15]:
import pandas as pd
import glob
from pathlib import Path

# Use glob to find all Parquet files in the folder
parquet_files = parquet_train_path.glob('*.parquet')

# Read and concatenate all Parquet files into a single DataFrame
df = pd.concat([pd.read_parquet(file) for file in parquet_files], ignore_index=True)

# # Display the first few rows of the DataFrame
# print(df.head())

# Get the total number of rows (games) in the DataFrame
total_games = len(df)
print(f"Total number of games in the dataset: {total_games}")

# Additional checks and summary statistics
print("Null values in each column:")
print(df.isnull().sum())

print("\nSummary statistics:")
print(df.describe())

# Count the number of unique words or game states
unique_words = df['word'].nunique()
print(f"\nNumber of unique words: {unique_words}")

# Inspect the distribution of game outcomes, difficulties, etc.
print("\nOutcome distribution:")
print(df['outcome'].value_counts())

print("\nDifficulty distribution:")
print(df['difficulty'].value_counts())


# Word length distribution
print("\nWord Length Distribution:")
print(df['word_length'].value_counts())

Total number of games in the dataset: 6894
Null values in each column:
game_id            0
word               0
initial_state      0
final_state        0
guessed_states     0
guessed_letters    0
game_state         0
difficulty         0
outcome            0
word_length        0
won                0
dtype: int64

Summary statistics:
           game_id  word_length
count  6894.000000  6894.000000
mean   3509.126632    12.113142
std    2004.567227     5.478361
min      42.000000     1.000000
25%    1765.250000     8.000000
50%    3512.500000    11.000000
75%    5247.750000    16.000000
max    6971.000000    29.000000

Number of unique words: 165

Outcome distribution:
outcome
win     3447
lose    3447
Name: count, dtype: int64

Difficulty distribution:
difficulty
easy      2298
medium    2298
hard      2298
Name: count, dtype: int64

Word Length Distribution:
word_length
8     588
9     588
10    546
12    504
11    462
7     462
6     378
13    336
15    294
17    294
18    294
5     2

In [16]:
import pandas as pd

# # Replace this with the path to your Parquet file
# parquet_file_path = 'path/to/your/HangmanData.parquet'

# Read the Parquet file
df = pd.read_parquet(parquet_train_path)

# Display the first few rows of the DataFrame
df

,game_id,word,initial_state,final_state,guessed_states,guessed_letters,game_state,difficulty,outcome,word_length,won
0,3150,millicurie,__________,millicurie,"__________,_____c____,_i__ic__i_,mi__ic__i_,mi...","c,i,m,l,d,b,j,u,r,e",allMasked,easy,win,10,True
1,3151,millicurie,__________,millicurie,"__________,_________e,_____c___e,__ll_c___e,__...","e,c,l,r,m,j,z,g,u,i",allMasked,easy,lose,10,True
2,3152,millicurie,__________,millicurie,"__________,______u___,______u__e,_____cu__e,__...","u,e,c,l,i,h,z,r,j,t,f,m",allMasked,medium,win,10,True
3,3153,millicurie,__________,m_____ur_e,"__________,_______r__,_______r__,______ur__,__...","r,n,u,x,g,m,h,e,b,w",allMasked,medium,lose,10,False
4,3154,millicurie,__________,__________,"__________,__________,__________,__________,__...","p,q,a,k,y,n",allMasked,hard,win,10,False
...,...,...,...,...,...,...,...,...,...,...,...
6889,6883,sick,_ick,sick,_ick,s,nearEnd,easy,lose,4,True
6890,6884,sick,_ick,sick,"_ick,_ick","q,s",nearEnd,medium,win,4,True
6891,6885,sick,_ick,sick,_ick,s,nearEnd,medium,lose,4,True
6892,6886,sick,_ick,_ick,"_ick,_ick,_ick,_ick,_ick,_ick","z,h,y,r,j,a",nearEnd,hard,win,4,False


In [17]:
# Get the total number of rows (games) in the DataFrame
total_games = len(df)

print(f"Total number of games in the dataset: {total_games}")

# Additional checks you might want to perform:
# - Check for any null values or anomalies in the data
print(df.isnull().sum())

# - Get a summary of the DataFrame
print(df.describe())

# - Count the number of unique words or game states
unique_words = df['word'].nunique()
print(f"Number of unique words: {unique_words}")

# - Inspect the distribution of game outcomes, difficulties, etc.
print(df['outcome'].value_counts())
print(df['difficulty'].value_counts())

Total number of games in the dataset: 6894
game_id            0
word               0
initial_state      0
final_state        0
guessed_states     0
guessed_letters    0
game_state         0
difficulty         0
outcome            0
word_length        0
won                0
dtype: int64
           game_id  word_length
count  6894.000000  6894.000000
mean   3509.126632    12.113142
std    2004.567227     5.478361
min      42.000000     1.000000
25%    1765.250000     8.000000
50%    3512.500000    11.000000
75%    5247.750000    16.000000
max    6971.000000    29.000000
Number of unique words: 165
outcome
win     3447
lose    3447
Name: count, dtype: int64
difficulty
easy      2298
medium    2298
hard      2298
Name: count, dtype: int64


##### Checking the Valid

In [18]:
import pandas as pd

# # Replace this with the path to your Parquet file
# parquet_file_path = 'path/to/your/HangmanData.parquet'

# Read the Parquet file
df = pd.read_parquet(parquet_valid_path)

# Display the first few rows of the DataFrame
df

,game_id,word,initial_state,final_state,guessed_states,guessed_letters,game_state,difficulty,outcome,word_length,won
0,462,gastroenterologically,_____________________,gastroenterologically,"_____________________,________________c____,__...","c,y,g,a,l,n,w,o,p,r,t,s,e,i",allMasked,easy,win,21,True
1,463,gastroenterologically,_____________________,gastroenterologically,"_____________________,____________l_____ll_,__...","l,y,f,e,o,c,g,d,h,a,s,t,q,r,n,i",allMasked,easy,lose,21,True
2,464,gastroenterologically,_____________________,_as__oen_e_o_o_ica__y,"_____________________,__s__________________,__...","s,x,a,n,c,m,o,d,v,z,e,i,y,w",allMasked,medium,win,21,False
3,465,gastroenterologically,_____________________,_a__roe__ero_o__ca___,"_____________________,________________c____,__...","c,q,o,e,j,r,z,d,x,a,f",allMasked,medium,lose,21,False
4,466,gastroenterologically,_____________________,_astr___t_r____ica___,"_____________________,_____________________,_a...","d,a,k,h,c,b,i,u,s,r,t,f",allMasked,hard,win,21,False
...,...,...,...,...,...,...,...,...,...,...,...
1759,79,noninstitution,nonin_titution,noninstitution,nonin_titution,s,nearEnd,easy,lose,14,True
1760,80,noninstitution,nonin_titution,noninstitution,"nonin_titution,nonin_titution","f,s",nearEnd,medium,win,14,True
1761,81,noninstitution,nonin_titution,noninstitution,"nonin_titution,nonin_titution,nonin_titution","l,r,s",nearEnd,medium,lose,14,True
1762,82,noninstitution,nonin_titution,noninstitution,"nonin_titution,nonin_titution","a,s",nearEnd,hard,win,14,True


In [19]:
# Get the total number of rows (games) in the DataFrame
total_games = len(df)

print(f"Total number of games in the dataset: {total_games}")

# Additional checks you might want to perform:
# - Check for any null values or anomalies in the data
print(df.isnull().sum())

# - Get a summary of the DataFrame
print(df.describe())

# - Count the number of unique words or game states
unique_words = df['word'].nunique()
print(f"Number of unique words: {unique_words}")

# - Inspect the distribution of game outcomes, difficulties, etc.
print(df['outcome'].value_counts())
print(df['difficulty'].value_counts())

Total number of games in the dataset: 1764
game_id            0
word               0
initial_state      0
final_state        0
guessed_states     0
guessed_letters    0
game_state         0
difficulty         0
outcome            0
word_length        0
won                0
dtype: int64
           game_id  word_length
count  1764.000000  1764.000000
mean    923.500000    14.023810
std     509.367255     5.268136
min      42.000000     4.000000
25%     482.750000     9.000000
50%     923.500000    14.000000
75%    1364.250000    18.000000
max    1805.000000    23.000000
Number of unique words: 42
outcome
win     882
lose    882
Name: count, dtype: int64
difficulty
easy      588
medium    588
hard      588
Name: count, dtype: int64


##### Reading Checking

In [20]:
# Create datasets directly from the saved parquet files
train_dataset = HangmanDataset(parquet_train_path)
valid_dataset = HangmanDataset(parquet_valid_path)

In [21]:
train_dataset[0]

{'game_id': 3150,
 'word': 'millicurie',
 'initial_state': ['__________'],
 'final_state': 'millicurie',
 'guessed_states': ['__________',
  '_____c____',
  '_i__ic__i_',
  'mi__ic__i_',
  'millic__i_',
  'millic__i_',
  'millic__i_',
  'millic__i_',
  'millicu_i_',
  'millicuri_'],
 'guessed_letters': ['c', 'i', 'm', 'l', 'd', 'b', 'j', 'u', 'r', 'e'],
 'game_state': 'allMasked',
 'difficulty': 'easy',
 'outcome': 'win',
 'word_length': 10,
 'won': False}

In [22]:
import os 

train_loader = DataLoader(train_dataset, batch_size=128, 
                          collate_fn=custom_collate_fn, 
                          shuffle=True, 
                          num_workers=os.cpu_count() or 1,  # Adjust based on your system
                          prefetch_factor=2)  # Adjust based on your needs
                          
val_loader = DataLoader(valid_dataset, batch_size=512, 
                          collate_fn=custom_collate_fn, 
                          shuffle=False, 
                          num_workers=os.cpu_count() or 1,  # Adjust based on your system
                          prefetch_factor=2)  # Adjust based on your needs

In [23]:
for batch in train_loader:
    pass